In [ ]:
import os

Hello


In [ ]:
os.chdir("../")
%pwd

Done


'c:\\Users\\tizia\\Documents\\Desktop\\MLOps\\datascienceproject'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [ ]:
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts])


